In [1]:
from jia_cidades import multi_regime, F, G, new_diagn, new_deaths, multi_regime_cidades, tot_deaths

In [2]:
from get_data import read_data, get_data, download_csv

In [3]:
from scipy.integrate import solve_ivp
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import matplotlib.ticker as mticker
import matplotlib.dates as mdates
import seaborn as sns
import pandas as pd
import unidecode
sns.set_style("darkgrid")

In [4]:
metropolitana_1 = ['RIO DE JANEIRO', 'ITAGUAI', 'SEROPÉDICA', 'QUEIMADOS', 'JAPERI', 'NOVA IGUAÇU', 'MESQUITA', 'NILOPOLIS', 'BELFORD ROXO', 'SAO JOAO DO MERITI', 'DUQUE DE CAXIAS', 'MAGE']

In [58]:
def read_data(df, fonte,tipo = 'confirmados'):
    if fonte == 'prefeitura':
        df  = df.drop(columns = ['sexo', 'classificação_final', 'dt_inicio_sintomas'])
        df.rename(mapper = {'faixa_etária' : 'Faixa Etaria', 'bairro_resid__estadia': 'Bairro',
                            'ap_residencia_estadia': 'AP Residência',
                               'dt_notific': 'Data', 'evolução': 'Situação atual'}, axis = 1, inplace = True)
        c = df.columns
        df[[c[1], c[-1]]] = df[[c[-1], c[1]]]
        df[[c[-2], c[-1]]] = df[[c[-1], c[-2]]]
        df[[c[2], c[3]]] = df[[c[3], c[2]]]
        df.rename(mapper = {'Bairro':'Situação atual', 'Situação atual':'Faixa etária',
                            'Faixa Etaria': 'AP Residência', 'AP Residência': 'Bairro'}, axis = 1, inplace = True)
        df = df.drop(labels = len(df)-1, axis = 0)
    if fonte == 'estado':
        if tipo == 'confirmados':
            df = df.drop(columns = ['sexo', 'uf', 'dt_sintoma', 'comorbidades', 'dt_obito'])
            df.rename(mapper = {'idade': 'Idade', 'municipio_res':'Município', 'dt_coleta / dt_notif':'Data',
                                            'evolucao':'Situação atual'}, axis = 1, inplace = True)
            c = df.columns
            df[[c[0],c[2]]] = df[[c[2],c[0]]]
            df[[c[1],c[-1]]] = df[[c[-1],c[1]]]
            df[[c[2],c[-1]]] = df[[c[-1],c[2]]]
            df.rename(mapper = {'Idade':'Data', 'Data':'Município', 'Município':'Situação atual',
                                                                  'Situação atual': 'Idade'}, axis = 1, inplace = True)
        elif tipo == 'obitos':
            c = df.columns
            df = df.drop(columns = [c[-1], c[-2], 'COMORBIDADE', 'SEXO', 'CONFIRMAÇÃO'])
            df.rename(mapper={'DIVULGAÇÃO':'Data', 'MUNICÍPIO':'Município', 'IDADE':'Idade'}, axis = 1, inplace = True)
            df['Município'] = [unidecode.unidecode(str(v).upper()) for v in df['Município']]
    return df

def get_data(local, df, fonte, T_fim, T_start = '29-03-2020'):
    if fonte == 'estado':
        if 'Situação atual' in df.columns:
            df_cidade = df.loc[df['Município'] == local]
            df_cidade = df_cidade.loc[df_cidade['Situação atual'] != 'NAO']
            n = len(df_cidade)
            k = n - len(df_cidade.dropna(subset=['Data']))
            df_cidade = df_cidade.dropna(subset=['Data'])
            print('No total ' + str(k) + ' dados foram inutilizados')
            l = list(df_cidade['Data'])
            start = dt.datetime.strptime(T_start, "%d-%m-%Y")
            then = dt.datetime.strptime(T_fim, "%d-%m-%Y")
            days = mdates.drange(start,then,dt.timedelta(days=1))
            dias = [str(mdates.num2date(v)).split(' ')[0] for v in days]
            dados_por_dia = len(dias)*[0]
            df_cidade_new = pd.to_datetime(df_cidade['Data'], dayfirst=True, format='%d/%m/%Y')
            times =list(df_cidade_new)
            times = [str(time).split(' ')[0] for time in times]
            for i, dia in enumerate(dias):
                dados_por_dia[i] += times.count(dia)
            dados = [sum(dados_por_dia[:i]) for i in range(1, len(dados_por_dia)+1)]
            acumulado_inicial = n - dados[-1]
            dados = [acumulado_inicial + dado for dado in dados]
            return dados, dados_por_dia
        else:
            df_cidade = df.loc[df['Município'] == local]
            n = len(df_cidade)
            df_cidade = df_cidade.dropna(subset=['Data'])
            l = list(df_cidade['Data'])
            start = dt.datetime.strptime(T_start, "%d-%m-%Y")
            then = dt.datetime.strptime(T_fim, "%d-%m-%Y")
            days = mdates.drange(start,then,dt.timedelta(days=1))
            dias = [str(mdates.num2date(v)).split(' ')[0] for v in days]
            dados_por_dia = len(dias)*[0]
            df_cidade_new = pd.to_datetime(df_cidade['Data'], dayfirst=True, format='%d/%m/%Y')
            times =list(df_cidade_new)
            times = [str(time).split(' ')[0] for time in times]
            for i, dia in enumerate(dias):
                dados_por_dia[i] += times.count(dia)
            dados = [sum(dados_por_dia[:i]) for i in range(1, len(dados_por_dia)+1)]
            acumulado_inicial = n - dados[-1]
            dados = [acumulado_inicial + dado for dado in dados]
            return dados, dados_por_dia
    elif fonte == 'prefeitura':
        df_bairro = df.loc[df['Bairro'] == local]
        n = len(df_bairro)
        k = n - len(df_bairro.dropna(subset=['Data']))
        df_bairro = df_bairro.dropna(subset=['Data'])
        print('No total ' + str(k) + ' dados foram inutilizados')
        l = list(df_bairro['Data'])
        start = dt.datetime.strptime(T_start, "%d-%m-%Y")
        then = dt.datetime.strptime(T_fim, "%d-%m-%Y")
        days = mdates.drange(start,then,dt.timedelta(days=1))
        dias = [str(mdates.num2date(v)).split(' ')[0] for v in days]
        dados_por_dia = len(dias)*[0]
        df_bairro_new = pd.to_datetime(df_bairro['Data'], dayfirst=True, format='%d/%m/%Y')
        times =list(df_bairro_new)
        times = [str(time).split(' ')[0] for time in times]
        for i, dia in enumerate(dias):
            dados_por_dia[i] += times.count(dia)
        dados = [sum(dados_por_dia[:i]) for i in range(1, len(dados_por_dia)+1)]
        acumulado_inicial = n - dados[-1]
        dados = [acumulado_inicial + dado for dado in dados]
        return dados, dados_por_dia

In [59]:
#df_conf_prf

In [63]:
get_data('', df_prf, T_fim = '15-04-2020', fonte = 'prefeitura')

No total 2 dados foram inutilizados


([8, 8, 8, 8, 9, 9, 9, 10, 12, 12, 12, 13, 16, 16, 17, 17, 18],
 [1, 0, 0, 0, 1, 0, 0, 1, 2, 0, 0, 1, 3, 0, 1, 0, 1])

In [42]:
df_conf_est

,sexo,idade,municipio_res,uf,dt_sintoma,dt_coleta / dt_notif,evolucao,dt_obito,comorbidades
0,M,68.0,TERESOPOLIS,RJ,24/03/2020,02/04/2020,OBITO,14/04/2020,SIM
1,M,88.0,RIO DE JANEIRO,RJ,07/04/2020,08/04/2020,OBITO,15/04/2020,SI
2,M,37.0,MESQUITA,RJ,29/03/2020,30/03/2020,OBITO,01/04/2020,SIM
3,M,53.0,RIO DE JANEIRO,RJ,09/04/2020,09/04/2020,OBITO,10/04/2020,SIM
4,F,85.0,RIO DE JANEIRO,RJ,26/03/2020,04/04/2020,OBITO,08/04/2020,NAO
...,...,...,...,...,...,...,...,...,...
5547,F,56.0,NITEROI,NaN,NaN,21/04/2020,NaN,NaN,NaN
5548,M,53.0,RIO DE JANEIRO,RJ,NaN,12/04/2020,NaN,NaN,NaN
5549,F,53.0,RIO DE JANEIRO,RJ,NaN,09/04/2020,NaN,NaN,NaN
5550,F,65.0,RIO DE JANEIRO,RJ,20/03/2020,11/04/2020,NaN,NaN,NaN


In [24]:
arquivo_conf_prf = 'data_municipios/dados_prefeitura_rio/Dados_indiv_MRJ_covid19_22-4-2020.csv'
arquivo_obito_est = 'data_municipios/dados_estado_rio/ÓbitosASCOM.csv'
arquivo_conf_est = 'data_municipios/dados_estado_rio/BICOVIDREAIS_22abrREV.csv'
df_conf_prf = pd.read_csv(arquivo_conf_prf, encoding = 'ISO-8859-1', delimiter = ';')
df_conf_est = pd.read_csv(arquivo_conf_est, delimiter = ';')
df_obitos = pd.read_csv(arquivo_obito_est, encoding = 'ISO-8859-1', delimiter = ';')
#df_confirmados = read_data(df_confirmados, 'confirmados')
#df_obitos = read_data(df_obitos, 'obitos')

In [25]:
#df_conf_est

In [26]:
#df_prf = read_data(df_conf_prf, fonte = 'prefeitura')
#df_o = read_data(df_obitos, fonte = 'estado', tipo = 'obitos')
#df = read_data(df_conf_est,'estado', 'confirmados')
#df_o

In [32]:
get_data('NITEROI', df, 'estado', T_fim = '12-04-2020')

No total 16 dados foram inutilizados


([178, 187, 199, 201, 205, 210, 210, 211, 213, 217, 217, 225, 225, 228],
 [3, 9, 12, 2, 4, 5, 0, 1, 2, 4, 0, 8, 0, 3])

In [78]:
df_o = 

,Data,Município,Idade
0,19/03/2020,Miguel Pereira,63
1,19/03/2020,Niterói,69
2,21/03/2020,Petrópolis,65
3,23/03/2020,Rio de Janeiro,58
4,24/03/2020,Rio de Janeiro,71
5,24/03/2020,Rio de Janeiro,74
6,25/03/2020,Rio de Janeiro,81
7,25/03/2020,Rio de Janeiro,61
8,26/03/2020,Rio de Janeiro,60
9,27/03/2020,Volta Redonda,66


In [81]:
df_o['Município'][0].upper()

'MIGUEL PEREIRA'